# Predicting Survival on the Titanic (Kaggle Competition)
[Link to Competition on Kaggle]('https://www.kaggle.com/competitions/titanic')

Timothy Miller II

## Basics
* Set working directory
* Import basic packages
* Upload Test and Train Sets

In [1]:
# set working directory
import os

os.chdir('/Users/timothymiller/Documents/GitHub/Titanic')

In [2]:
# import numpy and pandas
import numpy as np
import pandas as pd

# import data 
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
# train data view
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# test data view
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

For our ML models, we will need to change the string data into numerical dummy variables to allow for the computer to analyze.

In [5]:
# import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# set LabelEncoder object
le = LabelEncoder()

# specify the columns that we need to encode 
cols_to_encode = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

# create duplicate 'tidy' objects for train / test data
train_tidy = train
test_tidy = test

# encode columns to numerical data via fit_transform function
train_tidy[cols_to_encode] = train_tidy[cols_to_encode].apply(lambda col: le.fit_transform(col))
test_tidy[cols_to_encode] = test_tidy[cols_to_encode].apply(lambda col: le.fit_transform(col))


In [6]:
# check for missing values in training data
train_tidy.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [7]:
# replacing missing age values with mean age values in train
train_mean_age = train_tidy['Age'].mean() 
train_tidy['Age'].fillna(train_mean_age, inplace=True)


In [8]:
# check for missing values in the testing data
test_tidy.isnull().sum()

PassengerId     0
Pclass          0
Name            0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            1
Cabin           0
Embarked        0
dtype: int64

In [9]:
# repalcing missing age and fare data with averages for respective column

test_mean_age = test_tidy['Age'].mean()
test_mean_fare = test_tidy['Fare'].mean()
test_tidy['Age'].fillna(test_mean_age, inplace=True)
test_tidy['Fare'].fillna(test_mean_fare, inplace=True)



In [10]:
# check to ensure there are no null values remaining
check1 = test_tidy.isnull().sum().sum() == 0
check2 = train_tidy.isnull().sum().sum() == 0
print(check1,check2) # should return two true vals

True True


In [11]:
# separate dependent and independent vars
X_train = train_tidy.drop('Survived', axis=1)
y_train = train_tidy['Survived']
X_test = test_tidy
# y_test is unknown

In [12]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,108,1,22.000000,1,0,523,7.2500,147,2
1,2,1,190,0,38.000000,1,0,596,71.2833,81,0
2,3,3,353,0,26.000000,0,0,669,7.9250,147,2
3,4,1,272,0,35.000000,1,0,49,53.1000,55,2
4,5,3,15,1,35.000000,0,0,472,8.0500,147,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,548,1,27.000000,0,0,101,13.0000,147,2
887,888,1,303,0,19.000000,0,0,14,30.0000,30,2
888,889,3,413,0,29.699118,1,2,675,23.4500,147,2
889,890,1,81,1,26.000000,0,0,8,30.0000,60,0


In [13]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

## Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
clf = LogisticRegression()

# train the model on the training data
clf.fit(X_train,y_train)

# create predictions
pred_lr = clf.predict(X_test)

# create a dataframe of predictions
pred_lr_df = pd.DataFrame(pred_lr, columns=['Survived'])

# include passenger id's
ids = test_tidy[['PassengerId']]
pred_lr_df = pd.concat([ids, pred_lr_df], axis=1)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
# ensure there are 418 predictions
pred_lr_df.shape

(418, 2)

In [16]:
# import predictions to csv to submit
pred_lr_df.to_csv('predictions.csv', index=False)

## MLPClassifier Neural Net

In [21]:
from sklearn.neural_network import MLPClassifier
# initialize MLPClassifer Model
mlp = MLPClassifier(hidden_layer_sizes=(10000,5000), max_iter=500)
mlp.fit(X_train, y_train)

# create object to store predictions of our MLPClassifier
pred = mlp.predict(X_test)

# create a dataframe of predictions
pred_df = pd.DataFrame(pred, columns=['Survived'])

# include passenger id's
ids = test_tidy[['PassengerId']]
pred_df = pd.concat([ids, pred_df], axis=1)

In [22]:
# ensure there are 418 values
pred_df.shape

(418, 2)

In [23]:
# import predictions to csv for submission
pred_df.to_csv('predictions.csv', index=False)



## Gradient Boosting Classifier

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

# create gradient boosting classifier model
gb = GradientBoostingClassifier(n_estimators=10000000, learning_rate=0.05, max_depth=5, random_state=2)

# fit model onto training data
gb.fit(X_train,y_train)

# create predictions
pred_gb = gb.predict(X_test)


# create a dataframe of predictions
pred_gb_df = pd.DataFrame(pred_gb, columns=['Survived'])

# include passenger id's
ids = test_tidy[['PassengerId']]
pred_gb_df = pd.concat([ids, pred_gb_df], axis=1)

In [25]:
# ensure there are 418 predictions
pred_gb.shape

(418,)

In [26]:
# import predictions to csv for submission
pred_gb_df.to_csv('predictions.csv', index=False)

## Decision Tree Classifier

In [95]:
from sklearn.tree import DecisionTreeClassifier

# create decision tree classifier model
dt = DecisionTreeClassifier(
    criterion='entropy', max_depth=None, min_samples_split=10, min_samples_leaf=2,
    max_features=None, random_state=2
    )

# fit decision tree model to training data
dt.fit(X_train, y_train)

# create predictions
pred_dt = dt.predict(X_test)

# create a dataframe of predictions
pred_dt_df = pd.DataFrame(pred_dt, columns=['Survived'])

# include passenger id's
ids = test_tidy[['PassengerId']]
pred_dt_df = pd.concat([ids, pred_dt_df], axis=1)


In [96]:
# ensure there are 418 predictions
pred_dt_df.shape

(418, 2)

In [97]:
# import predictions to csv for submission
pred_dt_df.to_csv('predictions.csv', index=False)

# Current Score

In [101]:
# insert highest guessing percent from kaggle
highest_perc = .77033
# calculate correct number of passengers guessed
correct = round(highest_perc*418)
print(f"Correctly guessed {correct} out 418 passengers")

Correctly guessed 322 out 418 passengers
